In [1]:
import pandas as pd
import numpy as np
import xarray as xr
from functools import reduce

In [2]:
df = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_area_per5years_maize.xlsx")
df.head()

,country,type,1970,1975,1980,1985,1990,1995,2000,2005,2010,2015,2020
0,Afghanistan,grass,3.023369e+07,3.044680e+07,3.038133e+07,3.032514e+07,3.029580e+07,3.037184e+07,3.032836e+07,3.013668e+07,2.999706e+07,2.987126e+07,2.988271e+07
1,Albania,grass,4.495849e+05,4.443882e+05,4.531682e+05,4.484197e+05,4.507903e+05,4.433360e+05,4.393314e+05,4.277520e+05,4.120940e+05,4.157380e+05,3.983570e+05
2,Algeria,grass,3.403268e+07,3.465415e+07,3.493022e+07,3.461107e+07,3.449291e+07,3.463362e+07,3.472871e+07,3.460918e+07,3.466375e+07,3.468194e+07,3.467324e+07
3,American Samoa,grass,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,Andorra,grass,2.336054e+04,2.414865e+04,2.395282e+04,2.380702e+04,2.369093e+04,2.356838e+04,2.355871e+04,2.354252e+04,2.356344e+04,2.436579e+04,2.511696e+04


In [2]:
gfrac = xr.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_new.nc", engine="netcdf4")
grapc = xr.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GRAPC_new.nc", engine="netcdf4")
luh_static = xr.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/gpw_v4_national_identifier_grid_rev11_5_min_finall.nc", engine="netcdf4")
garea = xr.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")

gfrac_ngfbfc = [element.strip() for element in gfrac.coords['NGFBFC'].data.astype('str').tolist()]
grapc_ngfbfc = [element.strip() for element in grapc.coords['NFBFC'].data.astype('str').tolist()]


In [3]:
luc_crop = pd.read_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/luc_agri_prop_crop_AM1_v3.xlsx")
crop_footprint = pd.read_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/crop_footprint_image_region_v2.xlsx")
intensity_v2 = pd.read_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/crop_intensity_image_region_v2.xlsx")
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")

In [4]:
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['ISO Country'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((2160, 4320), dtype="<U14") #rubah ke <U64

for i in range(2160):
    for j in range(4320):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"
            
country = xr.Dataset({"country": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": gfrac.coords["longitude"].to_numpy(), "latitude": gfrac.coords["latitude"].to_numpy()})

__________________________________________________________________________________________________________________________________________________________________________________

In [5]:
garea_ha = np.nan_to_num(np.multiply(garea['GAREA'].isel(time=slice(0, 11)).values, 100))
gfrac_area = np.zeros((17, 11, 2160, 4320), dtype="float32")
gfrac_2020 = gfrac.isel(time=slice(0, 11)) 

for i in range(17):
    for n in range(0, 11):
        gfrac_area[i][n] = np.nan_to_num(np.multiply(gfrac_2020['GFRAC_new'].isel(NGFBFC=i, time=n), garea_ha[n]))

gfrac_netcdf = xr.Dataset(
    coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy(),
        "country": country['country']
    })
coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, gfrac_area[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
gfrac_netcdf = gfrac_netcdf.assign(data_vars)
gfrac_netcdf

<xarray.Dataset>
Dimensions:                           (longitude: 4320, latitude: 2160, time: 11)
Coordinates:
  * longitude                         (longitude) float64 -180.0 ... 180.0
  * latitude                          (latitude) float64 89.96 89.88 ... -89.96
  * time                              (time) datetime64[ns] 1970-01-01 ... 20...
    country                           (latitude, longitude) <U14 'ocean' ... ...
Data variables: (12/17)
    grass                             (time, latitude, longitude) float32 0.0...
    Wheat                             (time, latitude, longitude) float32 0.0...
    Rice                              (time, latitude, longitude) float32 0.0...
    Maize                             (time, latitude, longitude) float32 0.0...
    Tropical cereals                  (time, latitude, longitude) float32 0.0...
    Other temperate cereals           (time, latitude, longitude) float32 0.0...
    ...                                ...
    Tropical roots & tubers           (time, latitude, longitude) float32 0.0...
    Sugar crops                       (time, latitude, longitude) float32 0.0...
    Oil & palm fruit                  (time, latitude, longitude) float32 0.0...
    Vegetables & fruits               (time, latitude, longitude) float32 0.0...
    Other non-food & luxury & spices  (time, latitude, longitude) float32 0.0...
    Plant based fibres                (time, latitude, longitude) float32 0.0...

In [ ]:
dfgfrac = gfrac_netcdf.isel(time=0).to_dataframe()
dfgfrac1 = gfrac_netcdf.isel(time=1).to_dataframe()
# dfgfrac2 = gfrac_netcdf.isel(time=2).to_dataframe()
# dfgfrac3 = gfrac_netcdf.isel(time=3).to_dataframe()
# dfgfrac4 = gfrac_netcdf.isel(time=4).to_dataframe()
# dfgfrac5 = gfrac_netcdf.isel(time=5).to_dataframe()
# dfgfrac6 = gfrac_netcdf.isel(time=6).to_dataframe()
# dfgfrac7 = gfrac_netcdf.isel(time=7).to_dataframe()
# dfgfrac8 = gfrac_netcdf.isel(time=8).to_dataframe()
# dfgfrac9 = gfrac_netcdf.isel(time=9).to_dataframe()
# dfgfrac10 = gfrac_netcdf.isel(time=10).to_dataframe()

In [ ]:
table = pd.pivot_table(dfgfrac, values=gfrac_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table1 = pd.pivot_table(dfgfrac1, values=gfrac_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

# table2 = pd.pivot_table(dfgfrac2, values=gfrac_ngfbfc,
                    #    index=["country"], columns=['time'], fill_value=0)  

# table3 = pd.pivot_table(dfgfrac3, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table4 = pd.pivot_table(dfgfrac4, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table5 = pd.pivot_table(dfgfrac5, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table6 = pd.pivot_table(dfgfrac6, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table7 = pd.pivot_table(dfgfrac7, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table8 = pd.pivot_table(dfgfrac8, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table9 = pd.pivot_table(dfgfrac9, values=gfrac_ngfbfc,
#                        index=["country"], columns=['time'], fill_value=0)  

# table10 = pd.pivot_table(dfgfrac10, values=gfrac_ngfbfc,
                    #    index=["country"], columns=['time'], fill_value=0)  

df_index = table.stack(level=0)
df_index1 = table1.stack(level=0)
# df_index2 = table2.stack(level=0)
# df_index3 = table3.stack(level=0)
# df_index4 = table4.stack(level=0)
# df_index5 = table5.stack(level=0)
# df_index6 = table6.stack(level=0)
# df_index7 = table7.stack(level=0)
# df_index8 = table8.stack(level=0)
# df_index9 = table9.stack(level=0)
# df_index10 = table10.stack(level=0)

data = pd.to_datetime(df_index.columns, format='%d/%m/%Y %H.%M.%S')
df_index.columns = data.year
data1 = pd.to_datetime(df_index1.columns, format='%d/%m/%Y %H.%M.%S')
df_index1.columns = data1.year
# data2 = pd.to_datetime(df_index2.columns, format='%d/%m/%Y %H.%M.%S')
# df_index2.columns = data2.year
# data3 = pd.to_datetime(df_index3.columns, format='%d/%m/%Y %H.%M.%S')
# df_index3.columns = data3.year
# data4 = pd.to_datetime(df_index4.columns, format='%d/%m/%Y %H.%M.%S')
# df_index4.columns = data4.year
# data5 = pd.to_datetime(df_index5.columns, format='%d/%m/%Y %H.%M.%S')
# df_index5.columns = data5.year
# data6 = pd.to_datetime(df_index6.columns, format='%d/%m/%Y %H.%M.%S')
# df_index6.columns = data6.year
# data7 = pd.to_datetime(df_index7.columns, format='%d/%m/%Y %H.%M.%S')
# df_index7.columns = data7.year
# data8 = pd.to_datetime(df_index8.columns, format='%d/%m/%Y %H.%M.%S')
# df_index8.columns = data8.year
# data9 = pd.to_datetime(df_index9.columns, format='%d/%m/%Y %H.%M.%S')
# df_index9.columns = data9.year
# data10 = pd.to_datetime(df_index10.columns, format='%d/%m/%Y %H.%M.%S')
# df_index10.columns = data10.year

df_index = df_index.reset_index()
df_index1 = df_index1.reset_index()
# df_index2 = df_index2.reset_index()
# df_index3 = df_index3.reset_index()
# df_index4 = df_index4.reset_index()
# df_index5 = df_index5.reset_index()
# df_index6 = df_index6.reset_index()
# df_index7 = df_index7.reset_index()
# df_index8 = df_index8.reset_index()
# df_index9 = df_index9.reset_index()
# df_index10 = df_index10.reset_index()

df_index.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index1.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index2.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index3.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index4.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index5.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index6.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index7.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index8.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index9.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
# df_index10.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)

frames = [df_index, df_index1]

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['country', 'IMAGE Classification']), frames)
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)

merged_df = merged_df.fillna(0)
merged_df.to_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_new.csv", index=False)

__________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
grapc_ton = np.zeros((17, 11, 2160, 4320), dtype="float32")
grapc_2020 = grapc.isel(time=slice(0, 11)) 

for i in range(17):
    for n in range(0, 11):
        grapc_ton[i][n] = np.nan_to_num(np.multiply(grapc_2020['grapc_new'].isel(NGFBFC=i, time=n), garea_ha[n]))

grapc_netcdf = xr.Dataset(
    coords={
        "time": pd.date_range(start='1970-01-01', end='2020-01-01', freq='5YS'),
        "latitude": grapc.coords["latitude"].to_numpy(),
        "longitude": grapc.coords["longitude"].to_numpy(),
        "country": country['country']
    })
coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, grapc_ton[i]) for i, ngfbc in enumerate(grapc_ngfbfc)
}
grapc_netcdf = grapc_netcdf.assign(data_vars)
grapc_netcdf

In [ ]:
dfgrapc = grapc_netcdf.isel(time=0).to_dataframe()
dfgrapc1 = grapc_netcdf.isel(time=1).to_dataframe()
dfgrapc2 = grapc_netcdf.isel(time=2).to_dataframe()
dfgrapc3 = grapc_netcdf.isel(time=3).to_dataframe()
dfgrapc4 = grapc_netcdf.isel(time=4).to_dataframe()
dfgrapc5 = grapc_netcdf.isel(time=5).to_dataframe()
dfgrapc6 = grapc_netcdf.isel(time=6).to_dataframe()
dfgrapc7 = grapc_netcdf.isel(time=7).to_dataframe()
dfgrapc8 = grapc_netcdf.isel(time=8).to_dataframe()
dfgrapc9 = grapc_netcdf.isel(time=9).to_dataframe()
dfgrapc10 = grapc_netcdf.isel(time=10).to_dataframe()

In [ ]:
table = pd.pivot_table(dfgrapc, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table1 = pd.pivot_table(dfgrapc1, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table2 = pd.pivot_table(dfgrapc2, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table3 = pd.pivot_table(dfgrapc3, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table4 = pd.pivot_table(dfgrapc4, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table5 = pd.pivot_table(dfgrapc5, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table6 = pd.pivot_table(dfgrapc6, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table7 = pd.pivot_table(dfgrapc7, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table8 = pd.pivot_table(dfgrapc8, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table9 = pd.pivot_table(dfgrapc9, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

table10 = pd.pivot_table(dfgrapc10, values=grapc_ngfbfc,
                       index=["country"], columns=['time'], fill_value=0)  

df_index = table.stack(level=0)
df_index1 = table1.stack(level=0)
df_index2 = table2.stack(level=0)
df_index3 = table3.stack(level=0)
df_index4 = table4.stack(level=0)
df_index5 = table5.stack(level=0)
df_index6 = table6.stack(level=0)
df_index7 = table7.stack(level=0)
df_index8 = table8.stack(level=0)
df_index9 = table9.stack(level=0)
df_index10 = table10.stack(level=0)


data = pd.to_datetime(df_index.columns, format='%d/%m/%Y %H.%M.%S')
df_index.columns = data.year
data1 = pd.to_datetime(df_index1.columns, format='%d/%m/%Y %H.%M.%S')
df_index1.columns = data1.year
data2 = pd.to_datetime(df_index2.columns, format='%d/%m/%Y %H.%M.%S')
df_index2.columns = data2.year
data3 = pd.to_datetime(df_index3.columns, format='%d/%m/%Y %H.%M.%S')
df_index3.columns = data3.year
data4 = pd.to_datetime(df_index4.columns, format='%d/%m/%Y %H.%M.%S')
df_index4.columns = data4.year
data5 = pd.to_datetime(df_index5.columns, format='%d/%m/%Y %H.%M.%S')
df_index5.columns = data5.year
data6 = pd.to_datetime(df_index6.columns, format='%d/%m/%Y %H.%M.%S')
df_index6.columns = data6.year
data7 = pd.to_datetime(df_index7.columns, format='%d/%m/%Y %H.%M.%S')
df_index7.columns = data7.year
data8 = pd.to_datetime(df_index8.columns, format='%d/%m/%Y %H.%M.%S')
df_index8.columns = data8.year
data9 = pd.to_datetime(df_index9.columns, format='%d/%m/%Y %H.%M.%S')
df_index9.columns = data9.year
data10 = pd.to_datetime(df_index10.columns, format='%d/%m/%Y %H.%M.%S')
df_index10.columns = data10.year

df_index = df_index.reset_index()
df_index1 = df_index1.reset_index()
df_index2 = df_index2.reset_index()
df_index3 = df_index3.reset_index()
df_index4 = df_index4.reset_index()
df_index5 = df_index5.reset_index()
df_index6 = df_index6.reset_index()
df_index7 = df_index7.reset_index()
df_index8 = df_index8.reset_index()
df_index9 = df_index9.reset_index()
df_index10 = df_index10.reset_index()


df_index.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index1.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index2.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index3.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index4.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index5.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index6.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index7.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index8.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index9.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)
df_index10.rename(columns={'level_1': "IMAGE Classification"}, inplace=True)


frames = [df_index, df_index1, df_index2, df_index3, df_index4, df_index5, df_index6, df_index7, df_index8, df_index9, df_index10]

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['country', 'IMAGE Classification']), frames)
merged_df = merged_df.replace([np.inf, -np.inf], np.nan)

merged_df = merged_df.fillna(0)
merged_df.to_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GRAPC_new.csv", index=False)